# Imagenet inference via TFLite

In [1]:
import numpy as np
import os

In [2]:
import urllib
import zipfile

In [3]:
import cv2

In [4]:
import tensorflow as tf

In [5]:
print('TensorFlow Version: %s' % tf.__version__)

TensorFlow Version: 1.15.0


- Download starter model

In [6]:
model_zip = 'mobilenet_v1_1.0_224_quant_and_labels.zip'
unzip_dir = '_starter_model'

In [7]:
if not os.path.exists(model_zip):
    urllib.request.urlretrieve(
        'https://storage.googleapis.com/download.tensorflow.org/models/tflite/mobilenet_v1_1.0_224_quant_and_labels.zip', 
        'mobilenet_v1_1.0_224_quant_and_labels.zip')

In [8]:
with zipfile.ZipFile(model_zip) as zf:
    zf.extractall(unzip_dir)

- Load labels

In [9]:
labels = np.array(open(os.path.join(unzip_dir, 'labels_mobilenet_quant_v1_224.txt')).read().splitlines())

- Load image

In [10]:
img_size = 224

In [11]:
img = cv2.imread('dog.png')

In [12]:
# BGR-RGB conversion -> Resize -> Crop
img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

sz = np.asarray(img2.shape)
if sz[0] < sz[1]:
    sz = sz*256/sz[0]
else:
    sz = sz*256/sz[1]
sz = np.round(sz).astype(np.int32)
img2 = cv2.resize(img2, (sz[1], sz[0]) )

s0 = (int)((img2.shape[0]-img_size)/2)
s1 = (int)((img2.shape[1]-img_size)/2)
img2 = img2[s0:s0+img_size, s1:s1+img_size]

In [13]:
# Normalization not necessary?
img2.astype(np.uint8)
img2 = np.expand_dims(img2, axis=0)

- Load model

In [14]:
tfl_path = os.path.join(unzip_dir, 'mobilenet_v1_1.0_224_quant.tflite')

In [15]:
interp = tf.lite.Interpreter(model_path=tfl_path)

In [16]:
for t in interp.get_tensor_details():
    print('name = %s | shape = %s' % (t['name'], t['shape']) )

name = MobilenetV1/Logits/AvgPool_1a/AvgPool | shape = [   1    1    1 1024]
name = MobilenetV1/Logits/Conv2d_1c_1x1/BiasAdd | shape = [   1    1    1 1001]
name = MobilenetV1/Logits/Conv2d_1c_1x1/Conv2D_bias | shape = [1001]
name = MobilenetV1/Logits/Conv2d_1c_1x1/weights_quant/FakeQuantWithMinMaxVars | shape = [1001    1    1 1024]
name = MobilenetV1/Logits/SpatialSqueeze | shape = [   1 1001]
name = MobilenetV1/Logits/SpatialSqueeze_shape | shape = [2]
name = MobilenetV1/MobilenetV1/Conv2d_0/Conv2D_Fold_bias | shape = [32]
name = MobilenetV1/MobilenetV1/Conv2d_0/Relu6 | shape = [  1 112 112  32]
name = MobilenetV1/MobilenetV1/Conv2d_0/weights_quant/FakeQuantWithMinMaxVars | shape = [32  3  3  3]
name = MobilenetV1/MobilenetV1/Conv2d_10_depthwise/Relu6 | shape = [  1  14  14 512]
name = MobilenetV1/MobilenetV1/Conv2d_10_depthwise/depthwise_Fold_bias | shape = [512]
name = MobilenetV1/MobilenetV1/Conv2d_10_depthwise/weights_quant/FakeQuantWithMinMaxVars | shape = [  1   3   3 512]
nam

## Inference

In [17]:
input = interp.tensor(interp.get_input_details()[0]["index"])
output = interp.tensor(interp.get_output_details()[0]["index"])

In [18]:
interp.allocate_tensors()

In [19]:
img0 = img2
for i in range(3):
    print(i)
    interp.set_tensor(interp.get_input_details()[0]['index'], img0)
    interp.invoke()
    
    print('Class = %d (%.3f)' % (np.argmax(output()), np.max(output())/255) )

    if i == 0:
        img0 = np.zeros(img2.shape).astype(np.uint8)
    if i == 1:
        img0 = np.ones(img2.shape).astype(np.uint8)
    
    interp.reset_all_variables()

0
Class = 201 (0.275)
1
Class = 112 (0.090)
2
Class = 112 (0.071)


## Show result

In [20]:
print('Class = %s (%.3f)' % (labels[np.argmax(output())], np.max(output())/255) )

Class = nematode (0.071)
